# Geometry optimization

We use DFTK and the [GeometryOptimization](https://github.com/JuliaMolSim/GeometryOptimization.jl/)
package to find the minimal-energy bond length of the $H_2$ molecule.
First we set up an appropriate `DFTKCalculator` (see AtomsCalculators integration),
for which we use the LDA model just like in the Tutorial for silicon
in combination with a pseudodojo pseudopotential (see Pseudopotentials).

In [1]:
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf")
calc = DFTKCalculator(;
    model_kwargs = (; functionals=LDA(), pseudopotentials),  # model_DFT keyword arguments
    basis_kwargs = (; kgrid=[1, 1, 1], Ecut=10)  # PlaneWaveBasis keyword arguments
)

DFTKCalculator(functionals=Xc(lda_x, lda_c_pw), pseudopotentials=PseudoFamily("dojo.nc.sr.pbe.v0_4_1.standard.upf"), Ecut=10, kgrid=[1, 1, 1])

Next we set up an initial hydrogen molecule within a box of vacuum.
We use the parameters of the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/)
and DFTK's AtomsBase integration to setup the hydrogen molecule.
We employ a simulation box of 10 bohr times 10 bohr times 10 bohr and a
pseudodojo pseudopotential.

In [2]:
using LinearAlgebra
using Unitful
using UnitfulAtomic

r0 = 1.4   # Initial bond length in Bohr
a  = 10.0  # Box size in Bohr

cell_vectors = [[a, 0, 0]u"bohr", [0, a, 0]u"bohr", [0, 0, a]u"bohr"]
h2_crude = periodic_system([:H => [0, 0, 0.]u"bohr",
                            :H => [r0, 0, 0]u"bohr"],
                           cell_vectors)

FlexibleSystem(H₂, periodicity = TTT):
    cell_vectors      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [       0,        0,        0]u"a₀")
    Atom(H,  [     1.4,        0,        0]u"a₀")


Finally we call `minimize_energy!` to start the geometry optimisation.
We use `verbosity=2` to get some insight into the minimisation.
With `verbosity=1` only a summarising table would be printed and with
`verbosity=0` (default) the minimisation would be quiet.

In [3]:
using GeometryOptimization
results = minimize_energy!(h2_crude, calc; tol_forces=2e-6, verbosity=2)
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -1.110567473329                   -0.83   0.80    6.0    3.73s
  2   -1.117171702912       -2.18       -1.88   0.80    1.0    1.59s
  3   -1.117247831338       -4.12       -2.64   0.80    1.0   73.7ms
  4   -1.117251062259       -5.49       -3.27   0.80    2.0   21.8ms
  5   -1.117251183328       -6.92       -4.03   0.80    1.0   19.7ms
  6   -1.117251190114       -8.17       -4.70   0.80    2.0   21.8ms
  7   -1.117251190140      -10.58       -5.30   0.80    1.0   31.5ms
  8   -1.117251190143      -11.59       -6.54   0.80    1.0   20.4ms
  9   -1.117251190143      -13.09       -6.91   0.80    2.0   22.9ms
 10   -1.117251190143      -14.51       -7.76   0.80    1.0   24.7ms
 11   -1.117251190143   +  -14.45       -8.56   0.80    1.0   20.7ms
n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   

Structure after optimisation (note that the atom has wrapped around)

In [4]:
results.system

FlexibleSystem(H₂, periodicity = TTT):
    cell_vectors      : [      10        0        0;
                                0       10        0;
                                0        0       10]u"a₀"

    Atom(H,  [-0.0431829, -5.65548e-12, 8.56583e-12]u"a₀")
    Atom(H,  [ 1.44318, -4.07645e-12, 1.18966e-11]u"a₀")


Compute final bond length:

In [5]:
rmin = norm(position(results.system[1]) - position(results.system[2]))
println("Optimal bond length: ", rmin)

Optimal bond length: 1.4863658683129257 a₀


Our results (1.486 Bohr) agrees with the
[equivalent tutorial from ABINIT](https://docs.abinit.org/tutorial/base1/).

> **Cell optimisations**
>
> Recent versions of [GeometryOptimization](https://github.com/JuliaMolSim/GeometryOptimization.jl/)
> support cell shape optimisations as well by passing `variablecell=true` to `minimize_energy!`.
> See the
> [GeometryOptimization documentation](https://juliamolsim.github.io/GeometryOptimization.jl/stable/examples/variablecell/)
> for an example.